# Lista 1

## Uczenie maszynowe i sztuczna inteligencja

* [regresja](https://pl.wikipedia.org/wiki/Regresja_(statystyka))
* [podobieństwo konsinusowe](https://en.wikipedia.org/wiki/Cosine_similarity)

## Zadanie 1 (10pt)

Napisz wykorzystując regresję liniową program, który na podstawie oceny filmów przez użytkowników będzie
starał się przewidzieć ocenę innych użytkowników. Jako dane wykorzystamy zbiór
[MovieLens Latest Datasets](https://grouplens.org/datasets/movielens/latest/).
Dokładnie wybierzemy mniejszy zbiór, pobierz plik
[ml-latest-small.zip](http://files.grouplens.org/datasets/movielens/ml-latest-small.zip").
Zadanie polega na wybraniu z pliku <em>ratings.csv</em> tych użytkowników ($\,{userId}$), którzy ocenili film
'Toy Story (1995)', który w tym pliku ma identyfikator '1' (patrz <em>movies.csv</em>).
W tym pliku osób takich jest $215$. Wtedy zgodnie z zapisem z wykładu
$x_{ij}$ będzie oceną $i$-tego użytkownika dla $i=0,\ldots, 214$, bo
taki jest nasz zbiór osób, które oceniły 'Toy Story' oraz $j$ będzie oceną $j$-tego filmu
dla $j=0,\ldots,m$. Jako $j$ można wybrać $\,{movieId}$ filmu, czyli
np. film o $\,{movieId}=42$ oceniony przez użytkownika $5$ (nie jest to $\,{userId}$, tylko
piąta osoba ze zbioru $215$ osób), który ocenił film jako np. $3.5$ wpisujemy $x[5, 42] = 3.5$.
Natomiast $y_i$ to ocena 'Toy Story' przez $i$-tego użytkownika. Zatem tworzymy macierz $X = [x_{ij}]$ oraz wektor $y_i$
gdzie $i=0,\ldots,215$ oraz $j=0,\ldots,m$. Dla tak przygotowanych danych wykonujemy:

* regresje liniową na całym zbiorze użytkowników dla $m=10,1000,10000$,
  czyli np. dla $m=10$ ignorujemy filmy o $\,{movieId} \gt 10$ i robimy regresje
  dla tak okrojonego zbioru ocen. Jakie dostajemy błędy. Pokaż na wykresie.
* podziel zbiór osób na tzw. zbiór treningowy oraz zbiór testowy np.
  weźmy $i=0,\ldots,200$ to będzie zbiór treningowy i na takim zbiorze osób wykonajmy regresje
  natomiast później sprawdzamy już dla całości ($215$). Zatem ostatnie
  $15$ ocen będziemy chcieli przewidzieć (zbiór testowy). Zrób przewidywanie dla
  $m=10,100,200,500,1000,10000$. Wyświetl wynik predykcji i wynik prawidłowy
  dla tych $15$ osób.

Przykład danych do regresji jakie otrzymujemy dla $m=10$ i $n=215$.

```
X = [[0.  4.  0.  0.  4.  0.  0.  0.  0.  0. ]
     [0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
     [0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
     [0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
     [0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
     [3.  0.  0.  0.  4.  0.  0.  0.  0.  0. ]
     [3.  3.  0.  0.  0.  2.  0.  0.  2.  0. ]
     [3.5 0.  0.  0.  0.  0.  0.  0.  5.  0. ]
     ...
y = [[4. ]
     [4. ]
     [4.5]
     [2.5]
     [4.5]
     [3.5]
     [4. ]
     [3.5]
     ...
```


In [38]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [70]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400


In [98]:
users_ts = ratings.loc[ratings['movieId'] == 1]\
                  .reset_index(drop=True)\
                  .drop(['movieId', 'rating', 'timestamp'], axis=1)
users_ts

,userId
0,1
1,5
2,7
3,15
4,17
...,...
210,606
211,607
212,608
213,609


In [125]:
def get_user(user_id, users):
    return int(users.loc[users['userId'] == user_id].index[0])

def get_data(ratings, users, m):
    X = np.zeros((len(users),m))
    y = np.zeros((len(users)))
    for index, row in ratings.iterrows():
        user_id = int(row['userId'])
        if user_id not in users['userId'].tolist():
            continue
        user = get_user(user_id, users)
        movie = int(row['movieId'])
        rating = row['rating']
        if movie+2 < m:
            if movie == 1:
                y[user] = rating
            else:
                X[user][movie-2] = rating

    return X, y

In [126]:
X, y = get_data(ratings, users_ts, 10)
X[:10]

array([[0. , 4. , 0. , 0. , 4. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [3. , 0. , 0. , 0. , 4. , 0. , 0. , 0. , 0. , 0. ],
       [3. , 3. , 0. , 0. , 0. , 2. , 0. , 0. , 0. , 0. ],
       [3.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [4. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 3. , 0. , 4. , 0. , 0. , 0. , 0. ]])

## Zadanie 2 (15pt)

Napisz system rekomendacji filmów. Systemy takie są wykorzystywane przez różne firmy np.
Netflix organizował konkurs na opracowanie algorytmu, który będzie przewidywał
ocenę użytkownika [Netflix Prize](https://en.wikipedia.org/wiki/Netflix_Prize).
W zadaniu tym zaimplementujemy podobny system, który jednak zamiast przewidywania będzie na
podstawie preferencji użytkownika rekomendował filmy, które najprawdopodobniej mu się spodobają.
Istnieje wiele sposobów, aby taki system napisać, dla zainteresowanych bardziej tematem proponuje
zobaczyć np. [Recommendation Systems](http://infolab.stanford.edu/~ullman/mmds/ch9.pdf).
W tym zadaniu wybierzemy w miarę prosty i łatwy do implementacji system rekomendacji.
Sformalizujmy problem. Załóżmy, że mamy macierz oceny, gdzie wiersze będą reprezentować
użytkowników a kolumny filmy np.
$$ \begin{array}{ccc}
    & \,{Matrix} & \,{Star Wars IV} \\
    \,{Alice} & 5 & 4 \\
    \,{Bob} & 0 & 1 \\
    \,{John} & 2 & 2 \\
    \,{Ada} & 5 & 5
   \end{array}$$
Patrząc na powyższą macierz widać, że kolumny pierwsza i druga mają podobne oceny stąd można wywnioskować, że
filmy Matrix i Star Wars IV są (według użytkowników) podobne do siebie, czyli jeśli komuś podobał się Matrix
to jest duża szansa, że będzie podobał mu się Star Wars i odwrotnie. Dlatego "podobieństwo" sformalizujemy przez wykorzystanie
[podobieństwa cosinusowego](https://en.wikipedia.org/wiki/Cosine_similarity), czyli
jeśli $x = (x_1, x_2, \dots, x_n)$ i $y = (y_1, y_2, \dots, y_n)$, wtedy:
$$x_1 y_1 + x_2 y_2 + \cdots + x_n y_n = x \cdot y = \lVert x \rVert \lVert y \rVert \cos(\theta)$$
Gdzie $\theta$ jest kątem między wektorami. Wygodniej będzie nam normalizować wektory. Wtedy
$x_1 y_1 + x_2 y_2 + \cdots + x_n y_n = \cos(\theta).$ Wtedy $\cos(\theta)$ reprezentuje
podobieństwo jednego wektora do drugiego, czyli jak bardzo jedne oceny są bliskie innym.

Dla naszej macierzy otrzymujemy

In [1]:
import numpy as np

In [2]:
x = np.array([[5,0,2,5], [4,1,2,5]]).T
x

array([[5, 4],
       [0, 1],
       [2, 2],
       [5, 5]])

Obliczamy normę wektorów kolumnowych

In [3]:
np.linalg.norm(x, axis=0)

array([7.34846923, 6.78232998])

Obliczamy znormalizowaną macierz

In [4]:
x/np.linalg.norm(x, axis=0)

array([[0.68041382, 0.58976782],
       [0.        , 0.14744196],
       [0.27216553, 0.29488391],
       [0.68041382, 0.73720978]])

Nasza własna ocena filmów Matrix - 4, Star Wars IV - 3 dla wygody zapisujemy jako wektor kolumnowy

In [5]:
y = [[4],[3]]
y

[[4], [3]]

Obliczamy podobieństwo cosinusowe z każdym użytkownikiem (skorzystamy z mnożenia macierzowego)

In [6]:
z=np.dot(x/np.linalg.norm(x, axis=0), np.array(y)/np.linalg.norm(y))
z

array([[0.89819175],
       [0.08846517],
       [0.39466277],
       [0.98665692]])

Normalizujemy otrzymany wektor (będzie on reprezentował nasz profil filmowy)

In [7]:
z/np.linalg.norm(z)

array([[0.64422929],
       [0.06345177],
       [0.28307243],
       [0.70768107]])

Teraz musimy obliczyć podobieństwo cosinusowe między naszym profilem a każdą kolumną macierzy, aby znaleźć takie filmy, które są podobne do naszego profilu. Sortujemy po otrzymanym podobieństwu i dostajemy rekomendacje. Wykorzystując nasz przykład, otrzymujemy


In [8]:
X = x/np.linalg.norm(x, axis=0)
Z = z/np.linalg.norm(z)
np.dot(X.T, Z)

array([[0.99690104],
       [0.99448407]])

Stąd dostaliśmy większą rekomendacje dla filmu Matrix (0.996) niż Star Wars IV (0.994). Co było oczywiście oczekiwane
skoro nasza ocena była Matrix - 4, a Star Wars IV - 3.

Możemy, teraz przejść do właściwej części zadania.

### Zadanie do wykoniania

Napisz system rekomendacji filmów który będzie wykorzystywał dane
[MovieLens Latest Datasets](https://grouplens.org/datasets/movielens/latest/).
Dokładnie mniejszy zbiór (który dodatkowo trochę jeszcze zmniejszymy). Pobierz plik
[ml-latest-small.zip](http://files.grouplens.org/datasets/movielens/ml-latest-small.zip).
Interesować, będą nas głównie dwa pliki movies.csv oraz ratings.csv. W pliku ratings.csv mamy
właściwie wszystkie dane, które będą nam potrzebne aby stworzyć macierz oceny. Ponieważ
nawet w tym przypadku otrzymana macierz będzie dość duża dlatego w trakcie wczytywania
danych z pliku proszę wziąć pod uwagę tylko te wiersze (z pliku ratings.csv) w których <b>movie_id</b> (druga kolumna)
jest mniejsze od 10000 (resztę ignorujemy). Dla aktualnych danych ze strony otrzymamy, wtedy macierz mniej więcej 611x9019.
Przykładowy wynik dla wybranego profilu filmowego:

```
# wektor my_ratings odpowiada wektorowi y z przykładu wyżej
my_ratings = np.zeros((9019,1))
my_ratings[2571] = 5      # patrz movies.csv  2571 - Matrix
my_ratings[32] = 4        # 32 - Twelve Monkeys
my_ratings[260] = 5       # 260 - Star Wars IV
my_ratings[1097] = 4
my_ratings_norm = my_ratings/np.linalg.norm(my_ratings)

...

# otrzymujemy wynik rekomendacji po posortowaniu
# (cos(θ), movies_id)
(0.8675507828468105, 260)
(0.8362098349303669, 2571)
(0.8227451213877744, 1196)
(0.8002349214247857, 1210)
(0.7458504689612442, 1097)
(0.7286029159733108, 32)
(0.7265369898748615, 1198)
(0.7095672455110477, 1240)
(0.7029872178855614, 1270)
```

Otrzymane reprezentacje wyświetl w postaci nazw filmów, korzystając z movies.csv. Uwaga: w rzeczywistych
danych otrzymamy dużą liczbę zer, nawet całe kolumny, wtedy dostaniemy wartości NaN przy dzieleniu!
Rozwiąż ten problem wykorzystując np.nan_to_num(...).

